In [22]:
import requests
from bs4 import BeautifulSoup

base_url = "https://valuation2022.durban.gov.za/FramePages/"
form_page_url = "SearchType.aspx"
form_action_url = "SearchType.aspx"
search_result_url = "SearchResult.aspx"

session = requests.session()

In [23]:
def extract_search_request_data(soup):
    post_data = {
        "__EVENTTARGET": "btnGo",
        "__EVENTARGUMENT": "",
    }
    input_fields = soup.findAll("input")

    for field in input_fields:
        post_data[field["name"]] = field["value"]

    select_name = soup.find("select")["name"]
    options = soup.findAll("option")

    # Make a dictionary of select options and remove the empty first option
    options = list(map(lambda option: option["value"], options))[1:3]

    selected_option = options[0]
    post_data[select_name] = selected_option
    post_data.pop("btnGo", None)

    return post_data

In [24]:
def get_results(data):
    post_response = session.post(base_url + form_action_url, data=data, verify=False)

    # Check if a redirect happens
    if post_response.status_code == 302 or "Location" in post_response.headers:
        redirect_url = post_response.headers["Location"]

        search_response = session.get(redirect_url)
        results_html = search_response.text
        search_results_soup = BeautifulSoup(results_html, "html.parser")
    else:
        search_results_soup = BeautifulSoup(post_response.text, "html.parser")

    parameters = "?Roll=1&VolumeNo=&RateNumber=534&StreetNo=&StreetName=&Suburb=&ERF=&Portion=&DeedsTown=&SchemeName=&SectionNumber=&All=false"
    search_data_response = session.get(
        base_url + search_result_url + parameters, verify=False
    ).content

    search_data_soup = BeautifulSoup(search_data_response, "html.parser")
    results_table = search_data_soup.find("table")

    # Initialize a list to store each row's data
    data = []

    # Extract table headings
    headers = [header.text.strip() for header in results_table.find_all("th")]

    for row in results_table.find_all("tr"):
        columns = row.find_all("td")
        if columns:
            data.append({headers[i]: col.text.strip() for i, col in enumerate(columns)})

    return data

In [25]:
import pandas as pd

def clean_data(data):
    # Load data into a DataFrame
    df = pd.DataFrame(data)

    # Remove non-numeric "Rate Number" entries
    df = df[df['Rate Number'].str.isnumeric()]

    # Normalize column names (convert to lowercase, replace spaces with underscores)
    df.columns = df.columns.str.lower().str.replace(" ", "_")

    # Convert "registered_extent" to numeric, handling commas and empty strings
    df['registered_extent'] = pd.to_numeric(df['registered_extent'].str.replace(",", ""), errors='coerce')

    # Replace empty strings with NaN (which maps to NULL in PostgreSQL)
    df.replace({"": None}, inplace=True)
    
    return df

In [26]:
from sqlalchemy import create_engine

def store_data_to_db(dataframe, db_name):
    # Create SQLite database
    engine = create_engine(f'sqlite:///{db_name}.db')

    # Insert data into SQLite
    dataframe.to_sql('properties', engine, if_exists='replace', index=False)

    print("Data successfully inserted into SQLite database.")

In [27]:
import sqlite3

def get_data_from_db(db_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(f'{db_name}.db')

    # Query the database
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM properties")
    rows = cursor.fetchall()

    # Close the connection
    conn.close()
    
    return rows

In [28]:
# Get form page HTML
html = session.get(base_url + form_page_url, verify=False).content
# Prettify HTML output
mainPageSoup = BeautifulSoup(html)

search_form_data = extract_search_request_data(mainPageSoup)
result_data = get_results(search_form_data)

c:\Users\CHARL\.virtualenvs\challenge-scrap-Jm7KxQm2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valuation2022.durban.gov.za'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\CHARL\.virtualenvs\challenge-scrap-Jm7KxQm2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valuation2022.durban.gov.za'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\CHARL\.virtualenvs\challenge-scrap-Jm7KxQm2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valuation2022.durban.gov.za'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.i

In [29]:
dataframe = clean_data(result_data)
dataframe.head()

,rate_number,legal_description,address,first_owner,use_code,rating_category,market_value,registered_extent
2,96150534,ERF 18288 of PINETOWN,"10 211589 STREET, PINETOWN",RAJKUMAR DAYANAND,Vacant Land (47),Vacant Land,"50,000",399
3,51534286,ERF 195 of ELANGENI,"19 400356, HAMMARSDALE",BLACK S TRANSPORT AND LOGISTICS CC,Vacant Land (47),Vacant Land,"33,000",1084
4,65345130,ERF 393 of KLAARWATER,"13 40533 STREET, KLAARWATER",NENE PETROS,Single Residential (15),Residential,"240,000",425
5,43534177,ERF 65 of MPUMALANGA H,"35 645 STREET, MPUMALANGA-H",NTSHANGASE MZIKAYIFANI JOSEPH,Single Residential (15),Residential,"450,000",389
6,53400055,ERF 2280 of LOVU,"11 10404 STREET, LOVU",MEMELA XOLISILE JENNIFER,Single Residential (15),Residential,"150,000",301


In [30]:
db_name = "properties"
store_data_to_db(dataframe, db_name)

Data successfully inserted into SQLite database.


In [ ]:
rows = get_data_from_db(db_name)
rows

[('96150534',
  'ERF 18288 of PINETOWN',
  '10 211589 STREET, PINETOWN',
  'RAJKUMAR DAYANAND',
  'Vacant Land (47)',
  'Vacant Land',
  '50,000',
  399),
 ('51534286',
  'ERF 195 of ELANGENI',
  '19 400356, HAMMARSDALE',
  'BLACK S TRANSPORT AND LOGISTICS CC',
  'Vacant Land (47)',
  'Vacant Land',
  '33,000',
  1084),
 ('65345130',
  'ERF 393 of KLAARWATER',
  '13 40533 STREET, KLAARWATER',
  'NENE PETROS',
  'Single Residential (15)',
  'Residential',
  '240,000',
  425),
 ('43534177',
  'ERF 65 of MPUMALANGA H',
  '35 645 STREET, MPUMALANGA-H',
  'NTSHANGASE MZIKAYIFANI JOSEPH',
  'Single Residential (15)',
  'Residential',
  '450,000',
  389),
 ('53400055',
  'ERF 2280 of LOVU',
  '11 10404 STREET, LOVU',
  'MEMELA XOLISILE JENNIFER',
  'Single Residential (15)',
  'Residential',
  '150,000',
  301),
 ('53408136',
  'ERF 2304 of LOVU',
  '20 10405 STREET, LOVU',
  'NZIMANDE SIBONELO TRUEMAN',
  'Vacant Land (47)',
  'Vacant Land',
  '100,000',
  435),
 ('53416317',
  'ERF 2374 of 